## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
import pandas as pd

In [ ]:
def iterrows():
    summ = 0
    cnt = 0
    for index, row in reviews.iterrows():
        if '2011-01-01' > row.date >= '20010-01-01':
            cnt += 1
            summ += row['rating']
    return summ / cnt

def iterrows2010():
    summ = 0
    for index, row in reviews_2010.iterrows():
        summ += row['rating']
    return summ / reviews_2010.shape[0]

def seriesmean():
    return reviews_2010.mean()

reviews = pd.read_csv('/content/reviews_sample.csv', delimiter=',', index_col=0)
reviews_2010 = reviews[(reviews.date >= '20010-01-01') & (reviews.date <'2011-01-01')]

In [ ]:
%time
print(iterrows())

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
4.5230757650702165


In [ ]:
%time
print(iterrows2010())

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
4.5230757650702165


In [ ]:
%time
print(seriesmean())

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
!pip install line_profiler
%load_ext line_profiler
def iterrows2010():
    summ = 0
    for index, row in reviews_2010.iterrows():
        summ += row['rating']
    return summ / reviews_2010.shape[0]

print(iterrows2010())
%lprun -f iterrows2010 iterrows2010()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
4.5230757650702165


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
!pip install line_profiler
%load_ext line_profiler
def get_word_reviews_cnt(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

reviews = pd.read_csv('/content/reviews_sample.csv', delimiter=',', index_col=0)
%lprun -f get_word_reviews_count get_word_reviews_cnt(reviews)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
def MAPE1(recipe):
    absolute_deviation = 0
    reviews_recipe = reviews[reviews['recipe_id']==recipe]
    n = len(reviews_recipe)
    mean_rating = reviews_recipe.rating.sum() / n
    for name, value in reviews_recipe[['rating']].iteritems():
        absolute_deviation += abs(value - mean_rating)
    mape = absolute_deviation/n/mean_rating*100
    return mape

reviews = pd.read_csv('/content/reviews_sample.csv', delimiter=',', index_col=0)
print(MAPE1(21752))

960803    0.403226
960847    3.629032
960848    3.629032
960801    9.274194
960799    3.629032
960800    3.629032
960797    9.274194
960798    3.629032
Name: rating, dtype: float64


<ipython-input-7-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():


In [ ]:
import numpy as np
import numba

@numba.jit(nopython=True)
def MAPE2(recipe, ratings):
    absolute_deviation = 0
    count = 0
    reviews_recipe = ratings[ratings['recipe_id']==recipe]
    n = len(reviews_recipe)

    if n == 0:
        return 0

    mean_rating = reviews_recipe.rating.sum() / n
    for index, row in reviews_recipe.iterrows():
        rating = row['rating']
        if rating == 0:
            continue
        absolute_deviation += abs(rating - mean_rating)
        count += 1

    if count == 0:
        return 0

    mape = absolute_deviation / count / mean_rating * 100

    return mape

# преобразуем dataframe reviews в массив numpy
reviews_array = np.asarray('/content/reviews_sample.csv')

print(MAPE2(21752, reviews_array))

In [ ]:
def MAPE3(recipe):
    reviews_recipe = reviews[reviews['recipe_id'] == recipe].loc[reviews['rating'] != 0]
    n = len(reviews_recipe)
    
    if n == 0:
        return 0
    
    mean_rating = reviews_recipe.rating.mean()
    absolute_deviation = abs(reviews_recipe.rating - mean_rating).sum()
    mape = absolute_deviation / n / mean_rating * 100
    return mape
reviews = pd.read_csv('/content/reviews_sample.csv', delimiter=',', index_col=0)
print(MAPE3(21752))

37.096774193548384


In [ ]:
import numba
@numba.jit(nopython=True)
def MAPE4(recipe):
    reviews_recipe = reviews[reviews['recipe_id'] == recipe].loc[reviews['rating'] != 0]
    n = len(reviews_recipe)
    if n == 0:
        return 0
    mean_rating = reviews_recipe.rating.mean()
    absolute_deviation = np.abs(reviews_recipe.rating - mean_rating).sum()
    mape = absolute_deviation / n / mean_rating * 100
    return mape

In [ ]:
import timeit
repeat = 5
number = 100
t1 = timeit.Timer(lambda: MAPE1(21752))
time1 = t1.repeat(repeat=repeat, number=number)
print("MAPE1:", sum(time1) / len(time1))

<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b0

MAPE1: 0.22380111340007716


<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b06630d81e7c>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in reviews_recipe[['rating']].iteritems():
<ipython-input-10-b0

TypingError: ignored

In [ ]:
# Запускаем вторую реализацию
t2 = timeit.Timer(lambda: MAPE2(21752))
time2 = t2.repeat(repeat=repeat, number=number)
print("MAPE2:", sum(time2) / len(time2))



TypingError: ignored